# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [2]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('./data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [3]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index2word

['to',
 'you',
 'the',
 'and',
 'in',
 'is',
 'me',
 'my',
 'it',
 'for',
 'your',
 'of',
 'call',
 'have',
 'that',
 'on',
 'now',
 'are',
 'so',
 'can',
 'not',
 'but',
 'or',
 'at',
 'get',
 'do',
 'we',
 'be',
 'just',
 'with',
 'no',
 'if',
 'will',
 'ur',
 'this',
 'how',
 'lt',
 'gt',
 'what',
 'up',
 'when',
 'ok',
 'free',
 'go',
 'from',
 'out',
 'know',
 'll',
 'all',
 'like',
 'good',
 'got',
 'its',
 'then',
 'am',
 'come',
 'there',
 'time',
 'he',
 'day',
 'was',
 'only',
 'love',
 'send',
 'txt',
 'text',
 'want',
 'need',
 'going',
 'by',
 'one',
 'home',
 'as',
 'today',
 'she',
 'don',
 'stop',
 'still',
 'lor',
 'about',
 'da',
 'see',
 'back',
 'sorry',
 'our',
 'dont',
 'reply',
 'take',
 'tell',
 'mobile',
 'hi',
 'they',
 'her',
 'please',
 'here',
 'any',
 'week',
 'later',
 'been',
 'think',
 'dear',
 'pls',
 'some',
 'new',
 'did',
 'phone',
 'where',
 'well',
 're',
 'ì_',
 'who',
 'msg',
 'hey',
 'hope',
 'him',
 'much',
 'an',
 'great',
 'has',
 'had',
 'w

In [4]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index2word])
                     for ls in X_test])

<ipython-input-4-11a8d2e02a06>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index2word])


In [7]:
# Why is the length of the sentence different than the length of the sentence vector?
for i,v in enumerate(w2v_vect):
    print(len(X_test.iloc[i]), len(v))

19 19
8 3
23 18
8 7
18 18
7 7
2 2
6 6
7 6
22 22
6 6
8 7
10 10
4 3
10 10
6 5
5 3
16 15
6 6
21 21
22 20
11 11
68 50
8 8
14 14
22 20
7 7
7 6
11 11
9 9
6 6
9 7
34 33
21 16
8 7
20 18
38 33
11 10
8 7
23 19
18 15
25 25
6 6
6 5
16 15
28 25
5 5
26 26
7 7
26 26
21 20
9 8
6 6
23 13
24 24
27 23
19 16
4 4
26 24
24 22
7 7
5 4
8 8
7 7
20 19
6 4
4 4
25 24
30 30
8 7
9 8
21 18
18 14
22 17
25 23
25 22
30 30
24 21
12 10
50 43
15 15
9 9
10 9
13 12
5 5
28 24
19 17
22 22
6 5
6 5
3 3
7 7
11 10
7 6
4 1
6 6
3 2
23 23
14 14
6 6
20 18
30 25
35 27
7 5
7 7
1 1
6 6
39 36
38 24
9 9
20 19
4 4
16 12
9 9
36 22
13 10
18 15
4 4
9 9
5 4
6 6
13 11
7 7
5 4
18 16
15 15
19 19
24 19
9 9
4 2
7 7
12 12
12 10
6 6
10 8
2 2
1 1
7 7
24 23
25 25
19 19
14 13
25 20
1 1
25 25
8 8
7 7
10 10
6 5
21 20
19 18
16 15
10 8
8 8
15 14
6 6
10 10
8 8
28 24
26 21
6 6
6 6
27 25
7 6
8 8
25 23
18 17
7 5
54 38
22 19
11 9
11 11
18 16
21 21
6 6
23 23
13 12
9 9
4 2
17 16
23 17
11 11
25 24
4 4
5 5
15 13
21 20
30 26
14 13
7 7
5 5
17 15
5 5
12 11
19 17
12 10


In [10]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vect:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [11]:
# Are our sentence vector lengths consistent?
for i,v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]), len(v))

19 100
8 100
23 100
8 100
18 100
7 100
2 100
6 100
7 100
22 100
6 100
8 100
10 100
4 100
10 100
6 100
5 100
16 100
6 100
21 100
22 100
11 100
68 100
8 100
14 100
22 100
7 100
7 100
11 100
9 100
6 100
9 100
34 100
21 100
8 100
20 100
38 100
11 100
8 100
23 100
18 100
25 100
6 100
6 100
16 100
28 100
5 100
26 100
7 100
26 100
21 100
9 100
6 100
23 100
24 100
27 100
19 100
4 100
26 100
24 100
7 100
5 100
8 100
7 100
20 100
6 100
4 100
25 100
30 100
8 100
9 100
21 100
18 100
22 100
25 100
25 100
30 100
24 100
12 100
50 100
15 100
9 100
10 100
13 100
5 100
28 100
19 100
22 100
6 100
6 100
3 100
7 100
11 100
7 100
4 100
6 100
3 100
23 100
14 100
6 100
20 100
30 100
35 100
7 100
7 100
1 100
6 100
39 100
38 100
9 100
20 100
4 100
16 100
9 100
36 100
13 100
18 100
4 100
9 100
5 100
6 100
13 100
7 100
5 100
18 100
15 100
19 100
24 100
9 100
4 100
7 100
12 100
12 100
6 100
10 100
2 100
1 100
7 100
24 100
25 100
19 100
14 100
25 100
1 100
25 100
8 100
7 100
10 100
6 100
21 100
19 100
16 100
10 100